In [6]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])
y_pred


array([1])

In [20]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

import tensorflow.contrib.learn as tf

features_cols = tf.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.DNNClassifier(hidden_units=[300, 100], n_classes=10, feature_columns=features_cols)
dnn_clf = tf.SKCompat(dnn_clf)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)

W0718 10:25:38.177136  3900 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.
W0718 10:25:38.180136  3900 estimator.py:453] Using temporary folder as model directory: C:\Users\Lars\AppData\Local\Temp\tmpb3sxf8ou
W0718 10:25:38.181137  3900 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.


SKCompat()

In [33]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred["classes"])

W0718 10:37:16.262343  3900 data_feeder.py:283] float64 is not supported by many models, consider casting to float32.


0.9844444444444445

In [34]:
import tensorflow as tf

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=None, name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stdev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stdev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss=tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

W0718 11:12:29.234174  3900 deprecation.py:323] From <ipython-input-34-416982bd73a2>:25: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0718 11:12:29.236169  3900 deprecation.py:506] From c:\users\lars\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [35]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={
            X: mnist.validation.images,
            y: mnist.validation.labels
        })
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
        
    saver.save(sess, "./logs/my_model_final.ckpt")

W0718 11:20:00.280307  3900 deprecation.py:323] From <ipython-input-35-1f324459dc69>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0718 11:20:00.280307  3900 deprecation.py:323] From c:\users\lars\appdata\local\programs\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0718 11:20:00.282281  3900 deprecation.py:323] From c:\users\lars\appdata\local\programs\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Train accuracy: 0.88 Val accuracy: 0.903
1 Train accuracy: 0.9 Val accuracy: 0.9224
2 Train accuracy: 0.94 Val accuracy: 0.9316
3 Train accuracy: 0.94 Val accuracy: 0.9392
4 Train accuracy: 0.98 Val accuracy: 0.9446
5 Train accuracy: 0.94 Val accuracy: 0.9498
6 Train accuracy: 0.96 Val accuracy: 0.9518
7 Train accuracy: 0.94 Val accuracy: 0.9576
8 Train accuracy: 0.98 Val accuracy: 0.9606
9 Train accuracy: 1.0 Val accuracy: 0.9634
10 Train accuracy: 0.94 Val accuracy: 0.9644
11 Train accuracy: 0.98 Val accuracy: 0.967
12 Train accuracy: 1.0 Val 